# 人力推衍


* 透過數學模型，分析公司內人力隨遞移矩陣與聘僱品質的變動，直到平衡
* 此模型假設：
  * 遞移矩陣為常數，不隨公司人力比例、缺額數量等因素變動
  * 填補缺額的能力為常數，不隨公司人力比例、缺額數量等因素變動
  * 填補時只考慮當下的缺額總數，忽略該次迭代會產生的新缺額
  * 員工非好即壞
  * 人數使用浮點數
* 輸入參數若違反假設，可能會導致非預期的結果

## 參數定義

### 人力狀態 State

由於初始值對平衡態影響有限，人力狀態只接受一個輸入參數：HeadCount。

運算中會生成當時的 *好人* 與 *壞人* 數量，以矩陣形式儲存：

$
\begin{bmatrix}
Good \\
Bad
\end{bmatrix}
$

下式中的差額即為當時空缺：

$
HeadCount \geq Good + Bad
$

### 遞移矩陣 Transition

隨著時間的過去，員工可能在好壞之間轉變或離職。這個矩陣用來描述轉換發生的機率：

$
\begin{bmatrix}
Good-Good & Bad-Good\\
Good-Bad & Bad-Bad
\end{bmatrix}
$

機率合計會小於 1，差值即是離職率：

$
\begin{aligned}
GG + GB \leq 1 \\
BG + BB \leq 1
\end{aligned}
$

### 新聘任 NewHire

為模擬公司聘僱行為，補人只考慮當前缺額，忽略預期離職 (Transition)人數；
輸入參數為：

$
\begin{bmatrix}
PctGood \\
PctBad
\end{bmatrix}
$

聘用人數則為：

$
\begin{aligned}
\begin{bmatrix}
NewGood \\
NewBad
\end{bmatrix}
&= 
\begin{bmatrix}
PctGood \\
PctBad
\end{bmatrix}
* Vacancy \\
&= 
\begin{bmatrix}
PctGood \\
PctBad
\end{bmatrix}
* (HeadCount - Good - Bad)
\end{aligned}
$

## 程式碼

In [1]:
import numpy as np

In [2]:
class Tx:
    def __init__(self, GG, BG, GB, BB):
        self._tx = np.matrix([
            [GG, BG],
            [GB, BB]
        ])
        
    @property
    def tx(self):
        return np.matrix(self._tx)

    def __str__(self):
        return "Tranxition: {}".format(self._tx)

In [3]:
class NewHire:
    def __init__(self, PctGood, PctBad):
        self._nh = np.matrix([[PctGood], [PctBad]])
    
    @property
    def nh(self):
        return np.matrix(self._nh)

    def __str__(self):
        return "NewHire: {}".format(self._nh)

In [4]:
class Simulator2:
    def __init__(self, tx, nh, headCount=100):
        self._headCount = headCount
        self._state = np.matrix([[0], [0]])
        self._tx = tx
        self._nh = nh
        self._epoch = 0
        
    @property
    def vacancy(self):
        return self._headCount - self._state.sum()

    def step_forward(self):
        st = self._tx.tx * self._state
        if self.vacancy > 0:
            st += self._nh.nh * self.vacancy
        self._state = st
        self._epoch += 1    

    def stepping_report(self, iters, interval):
        print(self.report)
        for i in range(0, iters, interval):
            for _ in range(interval):
                self.step_forward()

            print(self.report)
    
    def evolve(self, epsilon=0.01, max_rounds=100):

        def roughly_equals(m1, m2, a1, a2):
            v1 = m1.item((a1, a2)) + epsilon
            v2 = m2.item((a1, a2))
            
            return abs((v2 - v1) / v1) <= epsilon
        
        while self._epoch < max_rounds:
            st = self._state
            self.step_forward()
            new_st = self._state
            
            if roughly_equals(st, new_st, 0, 0) and roughly_equals(st, new_st, 1, 0):
                print("Steady state: \n\t{}".format(self.report))
                return
        
        print("Not reaching stead state in {} rounds, \n\t{}".format(max_rounds, self.report))
            

    def __str__(self):
        return "Epoch: {}, \n{}, \n{}, \n{}".format(
            self._epoch,
            self._state,
            self._tx,
            self._nh,
        )
    
    @property
    def report(self):
        return "Epoch {}: (G:{:.2f}, B:{:.2f}), -{:.2f} Ppl".format(
            self._epoch, 
            self._state.item((0, 0)),
            self._state.item((1, 0)), 
            self.vacancy
        )

In [9]:
from collections import namedtuple
Conf = namedtuple('Conf', [
    'GG', 'GB', 'BG', 'BB', 
    'PctGood', 'PctBad', 
    'HeadCount'
])

def SimConf(conf):
    return Simulator2(
        tx=Tx(GG=conf.GG, GB=conf.GB, BG=conf.BG, BB=conf.BB),
        nh=NewHire(PctGood=conf.PctGood, PctBad=conf.PctBad),
        headCount=conf.HeadCount
    )

## 模擬

In [7]:
SimConf(Conf(
    GG=0.8, GB=0.05, 
    BG=0.01, BB=0.85, 
    PctGood=0.3, PctBad=0.05, 
    HeadCount=100)).evolve()

Steady state: 
	Epoch 15: (G:46.35, B:24.12), -29.53 Ppl


In [12]:
SimConf(Conf(
    GG=0.8, GB=0.05, 
    BG=0.01, BB=0.85, 
    PctGood=0.3, PctBad=0.3, 
    HeadCount=100)).evolve()

Steady state: 
	Epoch 10: (G:32.70, B:47.93), -19.37 Ppl
